In [20]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.datasets import mnist


In [21]:
#loading dataset
(x_train,y_train),(x_test,y_test)= mnist.load_data()

#Selecting 200 images
x_train = x_train[:200]
y=y_train[:200]

X=x_train.T #x_train.Transpose
X=X/255 #maximum number of pixels in an image is 255

y.resize((200,1))
y=y.T

pd.Series(y[0]).value_counts()

1    26
9    23
0    21
3    21
4    21
7    21
2    20
6    19
8    15
5    13
dtype: int64

In [22]:
# converting into binary classification As you can see,
#we have ten classes here – 0 to 9. 
#This is a multi-class classification problem. 
#For now, we will start with building a simple CNN model for a binary classification problem

for i in range(y.shape[1]):
    if y[0][i]> 4: #5-9 is labelled as 1
        y[0][i]=1
    else:
        y[0][i]=0 #0-4 is labelled as 0
        
pd.Series(y[0]).value_counts()
        



0    109
1     91
dtype: int64

In [23]:
#We will now initialize the filters for the convolution operation

f=np.random.uniform(size=(3,5,5))
f=f.T
print('Filter 1', '\n', f[:,:,0], '\n')
print('Filter 2', '\n', f[:,:,1], '\n')
print('Filter 3', '\n', f[:,:,2], '\n')


Filter 1 
 [[0.1978265  0.37951328 0.07437184 0.89313986 0.59331677]
 [0.9921053  0.71078108 0.6420034  0.9747461  0.65692554]
 [0.22389275 0.86856848 0.22651931 0.36966207 0.2305958 ]
 [0.25193526 0.71192794 0.97579266 0.64372124 0.37778196]
 [0.8409276  0.10585451 0.66116897 0.04335596 0.42404466]] 

Filter 2 
 [[0.30532489 0.50748864 0.17039    0.30319015 0.73062154]
 [0.79301253 0.43305302 0.81986337 0.88798695 0.59453829]
 [0.08420894 0.00735801 0.55211057 0.1045976  0.75457881]
 [0.85138245 0.73860578 0.8113237  0.56222024 0.79526409]
 [0.6332287  0.36727812 0.71201295 0.45675962 0.6177029 ]] 

Filter 3 
 [[0.98820802 0.37523123 0.8635067  0.21867458 0.72396345]
 [0.51442573 0.95977224 0.79057101 0.02618412 0.98919579]
 [0.69491355 0.35293309 0.22183293 0.16555789 0.87647262]
 [0.89525096 0.2273328  0.99213837 0.88315873 0.93928075]
 [0.30319731 0.97313542 0.1473612  0.05045513 0.33540267]] 



In [24]:
X.shape, y.shape, f.shape


((28, 28, 200), (1, 200), (5, 5, 3))

In [25]:
# Generating patches from images

new_image = []


# for number of images which is 200
for k in range(X.shape[2]):
    # sliding in horizontal direction
    for i in range(X.shape[0]-f.shape[0]+1):
        # sliding in vertical direction
        for j in range(X.shape[1]-f.shape[1]+1):
            new_image.append(X[:,:,k][i:i+f.shape[0],j:j+f.shape[1]])
          
            
new_image  = np.array(new_image)
new_image.resize((X.shape[2],int(new_image.shape[0]/X.shape[2]),new_image.shape[1],new_image.shape[2]))
new_image.shape

(200, 576, 5, 5)

In [26]:
#number of features in data set

s_row = X.shape[0] - f.shape[0] + 1
s_col = X.shape[1] - f.shape[1] + 1
num_filter = f.shape[2]

inputlayer_neurons = (s_row)*(s_col)*(num_filter)
output_neurons = 1 

# initializing weight
wo=np.random.uniform(size=(inputlayer_neurons,output_neurons))

In [27]:
# Defining the Sigmoid Function

def sigmoid(x):
    return 1/(1+ np.exp(-x))

# derivative of sigmoid function

def der_sigmoid(x):
    return x*(1-x)

In [28]:
# generating output of convo layer

filter_output = []

#for each image

for i in range(len(new_image)):
    #apply each filter
    for k in range(f.shape[2]):
        # # do element wise multiplication
         for j in range(new_image.shape[1]):
            filter_output.append((new_image[i][j]*f[:,:,k]).sum())
            
filter_output = np.resize(np.array(filter_output), (len(new_image),f.shape[2],new_image.shape[1]))

#applying activation over conv output
filter_output_sigmoid = sigmoid(filter_output)

filter_output.shape, filter_output_sigmoid.shape


((200, 3, 576), (200, 3, 576))

In [29]:
#Flattening the matrix
filter_output_sigmoid = filter_output_sigmoid.reshape((filter_output_sigmoid.shape[0],filter_output_sigmoid.shape[1]*filter_output_sigmoid.shape[2]))

filter_output_sigmoid = filter_output_sigmoid.T

# Linear trasnformation for fully Connected Layer
# Linear transformation for fully connectted layer

output_layer_input=np.dot(wo.T,filter_output_sigmoid)
output_layer_input = (output_layer_input - np.average(output_layer_input))/np.std(output_layer_input)


# activation function
output = sigmoid(output_layer_input)


In [30]:
#E = (y' - O)2/2
#∂E/∂O = -(y'-O)
#∂O/∂Z2 = (O)(1-O)
#∂Z2/∂W = A1

error= np.square(y-output)/2

error_wrt_output=-(y-output)
output_wrt_output_layer_input=output*(1-output)

output_wrt_w = filter_output_sigmoid



In [31]:

#delta change in w for fully connected layer
# W_new = W_old - lr*∂E/∂W
# ∂E/∂W = ∂E/∂O *∂O/∂Z2 * ∂Z2/∂W
delta_error_fcp = np.dot(output_wrt_w,(error_wrt_output * output_wrt_output_layer_input).T)
lr= 0.002
wo = wo - lr*delta_error_fcp


In [32]:
#derivatives for backpropagation for the convolutional layer and update the filters


#∂E/∂f = ∂E/∂O.∂O/∂Z2.∂Z2/∂A1 .∂A1/∂Z1 * ∂Z1/∂f
#∂E/∂O = -(y'-O)
#∂O/∂Z2 = (O)(1-O)
#∂Z2/∂A1 = WT
#∂A1/∂Z1= A1(1-A1)
#∂Z1/∂f = X

output_layer_input_wrt_filter_output_sigmoid = wo.T

#Error w.r.t sigmoid transformation
filter_output_sigmoid_wrt_filter_output = filter_output_sigmoid * (1-filter_output_sigmoid)

# cvalculating derivatives for backprop convolution
error_wrt_filter_output = np.dot(output_layer_input_wrt_filter_output_sigmoid.T,error_wrt_output*output_wrt_output_layer_input) * filter_output_sigmoid_wrt_filter_output
error_wrt_filter_output = np.average(error_wrt_filter_output, axis=1)
error_wrt_filter_output = np.resize(error_wrt_filter_output,(X.shape[0]-f.shape[0]+1,X.shape[1]-f.shape[1]+1, f.shape[2]))






In [33]:
filter_update = []
for i in range(f.shape[2]):
    for j in range(f.shape[0]):
        for k in range(f.shape[1]):            
            temp = 0
            spos_row = j
            spos_col = k
            epos_row = spos_row + s_row
            epos_col = spos_col + s_col
            for l in range(X.shape[2]):
                temp = temp + (X[spos_row:epos_row,spos_col:epos_col,l]*error_wrt_filter_output[:,:,i]).sum()
            filter_update.append(temp/X.shape[2])  

filter_update_array = np.array(filter_update)
filter_update_array = np.resize(filter_update_array,(f.shape[2],f.shape[0],f.shape[1]))

In [35]:
for i in range(f.shape[2]):
    f[:,:,i] = f[:,:,i] - lr*filter_update_array[i]